# Setup

In [ ]:
import numpy as np
import torch

print(f'{torch.version.git_version=}')
print(f'{torch.version.hip=}')
print(f'{torch.version.debug=}')
print(f'{torch.version.cuda=}')

cpu = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU {torch.cuda.current_device()}')
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
import h5py as h5

import tdg
import tdg.HMC as HMC
import tdg.plot as visualize

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

# Physics Parameters

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 5
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 8
beta = torch.tensor(1/25.)
mu = torch.tensor(-1.5*25.)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

# Markov Chain Parameters

What should we use to generate our Markov Chain?  In particular, if we use HMC we need to specify an integrator.

If you don't have a good guess for how many molecular dynamics steps you want, you might want to autotune.

In [ ]:
H = HMC.Hamiltonian(S)
A = HMC.Autotuner(H, HMC.Omelyan, cfgs_per_estimate=20)
integrator, start = A.target(0.75, start='hot', starting_md_steps=20, progress=tqdm)

fig, ax = plt.subplots(2,1, figsize=(8,6))
A.plot_history(ax[0])
A.plot_models(ax[1])
ax[1].legend()
fig.tight_layout()

# Production

Now we're ready to produce some configurations!

In [ ]:
configurations = 1000

In [ ]:
hmc = HMC.MarkovChain(H, integrator)
ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start=start, progress=tqdm)

In [ ]:
def plot_history(ensemble, history=None, label=None):
    
    if history is None:
        history = visualize.History(6)
    
    history.plot(ensemble.S.real,              0, x=ensemble.index, label=label)
    history.plot(ensemble.N.real,              1, x=ensemble.index, label=('fermionic' if not label else label))
    history.plot(ensemble.Kinetic.real,        2, x=ensemble.index, label=label)
    history.plot(ensemble.Potential.real,      3, x=ensemble.index, label=label)
    history.plot(ensemble.InternalEnergy.real, 4, x=ensemble.index, label=label)
    history.plot(ensemble.Contact.real,        5, x=ensemble.index, label=label)

    history.ax[0,0].set_ylabel('S')
    history.ax[1,0].set_ylabel('N')
    history.ax[2,0].set_ylabel('K')
    history.ax[3,0].set_ylabel('V')
    history.ax[4,0].set_ylabel('U')
    history.ax[5,0].set_ylabel('Contact')
    
    return history

In [ ]:
viz = plot_history(ensemble)

viz.plot(ensemble.N_bosonic.real, 1, label='bosonic'  )
viz.ax[1,1].legend()

In [ ]:
viz = visualize.ScatterTriangle(6,
    ('S', 'N', 'K', 'V', 'U', 'Contact'),
    )

viz.plot(
    (ensemble.S.real,
     ensemble.N.real,
     ensemble.Kinetic.real,
     ensemble.Potential.real,
     ensemble.InternalEnergy.real,
     ensemble.Contact.real,
    ))

We can compare the fermionic and bosonic number estimators.

In [ ]:
correlation = visualize.ScatterTriangle(2,
    ('fermionic', 'bosonic'))
correlation.plot((ensemble.N.real, ensemble.N_bosonic.real))
correlation.grid[1,0].plot([-5,15],[-5,15], linestyle=':', color='black')

# Storing to disk

We can write an ensemble to disk.

In [ ]:
storage='hmc.h5'

In [ ]:
with h5.File(storage, 'w') as f:
    ensemble.to_h5(f.create_group('/example'))

# Continuing Production

We can continue either from the ensemble already in memory or from the ensemble now on disk.

Let's compare the two methods.

To get the same results we must use the same random numbers.

In [ ]:
rng_state = torch.get_rng_state()
if torch.cuda.is_available():
    rng_state_gpu = torch.cuda.get_rng_state()

Now let's continue from what's in memory.

In [ ]:
torch.set_rng_state(rng_state)
if torch.cuda.is_available():
    torch.cuda.set_rng_state(rng_state_gpu)

from_memory = tdg.ensemble.GrandCanonical.continue_from(ensemble, configurations, progress=tqdm)

We can also continue from what's on disk.

In [ ]:
torch.set_rng_state(rng_state)
if torch.cuda.is_available():
    torch.cuda.set_rng_state(rng_state_gpu)

with h5.File(storage, 'r') as f:
    from_disk = tdg.ensemble.GrandCanonical.continue_from(f['/example'], configurations, progress=tqdm)

To compare, let's plot observables from the two continuations on top of one another and hope for perfect agreement.

In [ ]:
viz = plot_history(from_memory, label='from memory')
viz = plot_history(from_disk, history=viz, label='from disk')

viz.plot(from_memory.N_bosonic.real, 1, x=from_memory.index, label='from memory'  )
viz.plot(from_disk  .N_bosonic.real, 1, x=from_disk  .index, label='from disk'    )

viz.ax[0,0].legend()

Since they match, let's write one, appending it to the ensemble already on disk.

In [ ]:
with h5.File(storage, 'a') as f:
    from_disk.extend_h5(f['/example'])

We can check that the appending succeeded again by comparing observables.

In [ ]:
with h5.File(storage, 'r') as f:
    combined = tdg.ensemble.GrandCanonical.from_h5(f['/example'])
    
viz = plot_history(combined, label='combined on disk')
viz = plot_history(ensemble,    history=viz, label='original ensemble')
viz = plot_history(from_memory, history=viz, label='extension')

viz.ax[0,0].legend()